In [13]:
import pandas as pd
import numpy as np
import geopandas as gpd
import json
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

In [38]:
alb_mbd = pd.read_excel("../Data/Mali/Mali_JRSM_2018_02042020_1580811994.xlsm", sheet_name="ALB_MBD", skiprows=5)
pzq = pd.read_excel("../Data/Mali/Mali_JRSM_2018_02042020_1580811994.xlsm", sheet_name="PZQ", skiprows=5)
ivm = pd.read_excel("../Data/Mali/Mali_JRSM_2018_02042020_1580811994.xlsm", sheet_name="IVM", skiprows=5)
pop = pd.read_excel("../Data/Mali/Mali_JRSM_2018_02042020_1580811994.xlsm", sheet_name="COUNTRY_INFO", skiprows=5)

In [76]:
print(pop.columns)
print(pop.loc[0])
# print(alb_mbd.loc[1])
# #ivm.head()
# alb_mbd.head()

Index(['Structure administrative du pays', 'Unnamed: 1', 'Unnamed: 2',
       'Population', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Endémicité',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Population nécessitant la CP', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Nombre de tournées de traitement prévues pour l'année',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Enquêtes épidémiologiques planifiées pour l’année', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'],
      dtype='object')
Structure administrative du pays                                       Pays
Unnamed: 1                                               Région/Département
Unnamed: 2                                                         District
Population                                                            Total
Unnamed: 4                                                           EAPréS
Unnamed: 5                                               

In [44]:
#IVM LF & Oncho
ivm_total_treatment_lf = ivm.iloc[2:, 6]
ivm_total_treatment_oncho = ivm.iloc[2:, 9]
ivm_total_treatment = ivm_total_treatment_lf + ivm_total_treatment_oncho
ivm_total_procured = ivm.iloc[2:, 15] + ivm.iloc[2:, 16]

In [45]:
#Schistosoma
pzq_dist_names = pzq.iloc[2:, 2]
pzq_total_treatment = pzq.iloc[2:, 5]
pzq_tab_procured = pzq.iloc[2:, [16, 17]]
pzq_total_procured = pzq_tab_procured.iloc[:, 0] + pzq_tab_procured.iloc[:, 1]

In [77]:
# Here I selecte the different colummns I  need and store them as seperate Series. The rows start at level 2
# The columns for the first two variables have names, but the rest I find by indexing.
state_names = alb_mbd.loc[2:, "Unnamed: 1"]
population = pop.loc[2:, "Population"]
district_names = alb_mbd.loc[2:, "Unnamed: 2"]
total_treatment = alb_mbd.loc[2:, "Unnamed: 5"]
total_target_alb = alb_mbd.iloc[2:, 13]
total_target_mbd = alb_mbd.iloc[2:, 17]
lf_alb_tab_bottle = alb_mbd.iloc[2:, 19:21]
sac_alb_tab_bottle = alb_mbd.iloc[2:, 22:24]
sac_mbd_tab_bottle = alb_mbd.iloc[2:, 25:27]

In [78]:
# Checking to see if the data is actually selected
sac_mbd_tab_bottle.head(1)

,Unnamed: 25,Unnamed: 26
2,0,0


In [81]:
selectedData = pd.DataFrame({'state_names':state_names, 'district_names':district_names, 'population': population, 'total_treatment_alb_mbd':total_treatment, 'total_target_alb':total_target_alb, 'total_target_mbd':total_target_mbd, 'lf_alb_total_procured':lf_alb_tab_bottle.iloc[:,0] + lf_alb_tab_bottle.iloc[:,1], 'sac_alb_total_procured':sac_alb_tab_bottle.iloc[:,0] + sac_alb_tab_bottle.iloc[:,1], 'sth_mbd_total_procured':sac_mbd_tab_bottle.iloc[:,0] + sac_mbd_tab_bottle.iloc[:,1], 
                            'pzq_total_treatment':pzq_total_treatment, 'pzq_total_procured':pzq_total_procured, 'ivm_total_treatment':ivm_total_treatment, 'ivm_total_procured':ivm_total_procured})

for col in selectedData.columns:
    if col != 'state_names' and col != 'district_names':
        selectedData[col] = pd.to_numeric(selectedData[col])

# selectedData = selectedData.groupby('state_names').mean().reset_index()
selectedData.to_csv('../Data/Mali/filtered_mali_data1.csv', index=False)
selectedData.head(10)

,state_names,district_names,population,total_treatment_alb_mbd,total_target_alb,total_target_mbd,lf_alb_total_procured,sac_alb_total_procured,sth_mbd_total_procured,pzq_total_treatment,pzq_total_procured,ivm_total_treatment,ivm_total_procured
2,Kayes,Bafoulabe,172888,0,0,0,0,0,0,15974.97,39977.425,137446,385618.8
3,Kayes,Oussoubidiagna,141453,0,0,0,0,0,0,13070.31,32708.775,112455,315504.0
4,Kayes,Diema,284961,0,0,0,0,0,0,26330.37,65891.925,0,0.0
5,Kayes,Kayes,689836,0,0,0,0,0,0,63740.82,159512.050,548420,1538648.0
6,Kayes,Kenieba,260896,0,0,0,0,0,0,24106.83,60328.075,207412,581915.6
7,Kayes,Kita,464377,0,0,0,0,0,0,0.00,0.000,369180,1035772.0
8,Kayes,Sefeto,64841,0,0,0,0,0,0,0.00,0.000,51548,144623.4
9,Kayes,Sagabari,54485,0,0,0,0,0,0,0.00,0.000,43316,121527.8
10,Kayes,Nioro,309720,0,0,0,0,0,0,28618.26,71617.650,0,0.0
11,Kayes,Yelimane,239783,0,0,0,0,0,0,0.00,0.000,0,0.0
